In [ ]:
!pip install pyarrow==2.0.0
!pip install git+https://github.com/minimaxir/textgenrnn.git

     |████████████████████████████████| 17.7MB 205kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Cloning https://github.com/minimaxir/textgenrnn.git to /tmp/pip-req-build-2is8glc5
  Running command git clone -q https://github.com/minimaxir/textgenrnn.git /tmp/pip-req-build-2is8glc5
  Created wheel for textgenrnn: filename=textgenrnn-2.0.0-cp36-none-any.whl size=1734418 sha256=a38a379a46c412c66c718f91b326b61e3264d391e1947960e8979aecdeeccbaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-lcpwo57h/wheels/65/ee/bd/b3bd6d3d84e916279b403b1d83d6a6a820d0b47226ad92bc97
Successfully built textgenrnn
  Found existing installation: textgenrnn 1.4.1
    Uninstalling textgenrnn-1.4.1:
      Successfully uninstalled textgenrnn-1.4.1


In [ ]:
!pip install pygithub

     |████████████████████████████████| 296kB 14.9MB/s 


In [ ]:
import pandas as pd
import time
from github import Github

In [ ]:
github = Github("aefcbe45583e1ee87ddea62bad904e4b095d1455")
def push(repo_path, file_path):
  filename = f"{time.time():.0f}.{file_path.split('.')[-1]}"
  github.get_repo('Scytheface/NARV').create_file(f"{repo_path}{filename}",
                   requests.get('http://whatthecommit.com/index.txt').text.strip(),
                   open(file_path, 'rb').read())

In [ ]:
import os

if 'COLAB_TPU_ADDR' in os.environ:
    import tensorflow as tf
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
else:
    import contextlib
    strategy = type("", (), {'scope': contextlib.suppress})

In [ ]:
import requests
data = pd.read_feather('https://github.com/Scytheface/NARV/raw/main/lyrics/lyrics.ft')

In [ ]:
data.head()

,artist,title,lyrics
0,dvph,punased käed,[Intro]\nKes seal on?\n\n[Verse 1]\nDVPH - see...
1,mõmmi,modellid,(HOOK: MÕMMI)\nKallid rõivad\nNeed on seljas\n...
2,mõmmi,meeldib,[HOOK; MÕMMI]\nTalle meeldib playboy\nRanne jä...
3,mõmmi,mukehamavõpatan,"Luuad pakule, makule meeldib\nJoogid ju valatu..."
4,nublu,für oksana,"А это жизнь, это карма\nЭй, привет, город Нарв..."


In [ ]:
texts = []
for _, row in data.iterrows():
  if row['lyrics']:
    texts.append(row['lyrics'].lower())

In [ ]:
  from textgenrnn import textgenrnn

  model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 256,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
  }

  train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 100,   # set higher to train the model for longer
    'gen_epochs': 100,   # generates sample text from model after given number of epochs
    'train_size': 1,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
  }

  textgen = textgenrnn(name='tgrnn')
  
  textgen.train_on_texts(
      save_epochs=1,
    texts=texts,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])
  textgen.save('rapmodel.hdf5')
  #push('models/', 'rapmodel.hdf5')  
  #push('models/', 'tgrnn_weights_epoch_200.hdf5') 

Training new model w/ 5-layer, 256-cell LSTMs
Training on 1,783,307 character sequences.
Epoch 1/100


KeyboardInterrupt: ignored

In [ ]:
from textgenrnn import textgenrnn
tg = textgenrnn(config_path='tgrnn_config.json', weights_path='tgrnn_weights_epoch_30.hdf5', vocab_path='tgrnn_vocab.json')

In [ ]:
tg.generate(5, max_gen_length=1000
            )

 20%|██        | 1/5 [00:04<00:17,  4.37s/it]

ta tahtis teada ma viga . 
 
 ikka ju näen , et ka silmast ja su ema omaje tead ma sain , 
 kuid miski pillumist ja raudne kopp 
 oi king kong ! 



 40%|████      | 2/5 [00:34<00:36, 12.12s/it]

 [ intro ] 
 rahabos ja ma ei tea ta minu hoolekult on su kõrval 
 sinu jaoks on see tšort 
 see on paradoks kui narko surma tipp ajab riip 
 sa ei saa , mul kanal ei tee 
 aga tegelt ei saa 
 
 sa arvad kõik saab korda , 
 nii ma arvasin ka 
 sa arvad kõik saab korda , 
 aga tegelt ei saa 
 
 sa arvad kõik saab korda , 
 aga tegelt ei saa 
 
 sa arvad kõik saab korda , 
 aga tegelt ei saa 
 
 sa arvad kõik saab korda , üle sõita neiu 
 sest mu emme vaikselt armastan . 
 ma ei saa kunagi tungita lasta 
 ta tagaini seitse vana 
 naabrib kürjaks minu seljas , jook jeesus 
 tõsta klubis näinud , natukene veel 
 ja ma ei armasta mind ja mingi perse ! 
 kuhu tee kutsun ta 
 
 [ outro ] 
 võit - ja ( võidan viite ) 
 su nie [ ? ] , sul on suva all , juba ma palun andma 
 andke kohe mitte midagi teha 
 ma ei oska kaine peaga peol mitte midagi teha 
 ma ei saa sinu vastast enam mitte üksi 
 kuid ma pole lihtsalt pätt tindida 
 siin mängus saadab kõik mis ma läbi aknast 
 
 [ chorus ] 
 kui me


 60%|██████    | 3/5 [01:05<00:35, 17.67s/it]

jou ja me autos 
 sa tead seda enne kui telefoni 
 su libe kajab , aasta oma tööpäeva lõpuks ühest pole huvi 
 sinu südames on mu ununes 
 sa ei saagi aru 
 ja kõik on üks dark , kas õpetaja peidab nagu tarkust 
 naabris elus suvel näeb 
 kus käis on , seepilk ja see tähtis on meil sihin pikikuthev . 
 siin on kõike piisavalt , a sellest ei piisa . 
 limpsab alati uue järgi keelt . 
 liigsed valikud teevad ju liiga , 
 alistades siis see on 
 võõraste seleta 
 kuid ainult seal bansivad koolid 
 
 [ bridge : mard ] 
 kolm koma on sulle täpselt nii 
 sorry , et maa on tunne , 
 nagu me enda tulemast 
 
 beebilõust : 
 mu singel ei valluta ees , et tervika kallim mu meeles 
 sa tahad tasa dype ' s ongi gramm 
 lõpp on koksi siis nõme - lits kony 
 kätte lasen lahti ja kõigil on juppe 
 teen kõik asjadel astudes 
 ei tea mitu valmis hoopis 
 olen legendam rahutust läheb , 
 oma austu jälle nullis ! 
 
 “ mis saab siis ? 
 
 ma olen hustlik , oled parim , parem ja parem 
 ja kui sa vambis 


 80%|████████  | 4/5 [01:35<00:21, 21.45s/it]

ma elad söögin sinu imidokki 
 see on kantra 
 mu pasa miks sa magad mind maha 
 ei taha kedo maha kõik kes vaid vaabab 
 kui sulle ei meeldi sinusugust pole vaja 
 oma eluluhule andestage kui ta liha teed 
 mõtlen sind ja mina jään 
 veel üks peer 
 võtan viin ma 
 küllast unusta , mis asoonikuud ? 
 kaks vastupidi nagu oleks libistamist 
 mõned kui tuules see on ju õige 
 
 [ verse 3 ] 
 kui  teada sama palju tagasi ma saan 
 kuid nii ma pole tahtnud 
 kikivarvul kõnnin ümber palava pudru 
 nii ma , nii ma astun , 
 kuid nii ma pole tahtnud 
 kümned näed , et 
 sa jääd ka veel 
 kahe käia võiks vahest karjub 
 minut mõtlen aint sulle fakting läbi 
 mu kütted , ma tulen ja tunnen nagu inimese 
 ja kuula mu nime - sinu kohta 
 päikse lähen vahel palju on hea 
 see on feik , ei ole alistu 
 olen paitsnud kes see paneb emotsioon 
 kui ma kuulen sind naine nagu ma ja ma kannan su eluga 
 kus on minu sõnu 
 seest kas midagi muud , kuid see on jumala eos 
 
 2 . kõik on nii , et mis siin 



100%|██████████| 5/5 [02:06<00:00, 25.22s/it]

jou , ma loome loomist 
 kuidas kullid on siia 
 
 [ chorus ] 
 ja - a - a - a 
 ja - a - a - a - a - a - a - a - a - a - a - a 
 ja - aa - a - jaa , 
 
 jaja 
 igar m maja , su sebi on mu pane julge 
 
 verse : ees ja ma pole sees 
 
 sa tõid ju kodus ja kaotus 
 sinu jaoks ta näljane 
 maga on üks minu teine nagu ma teind 
 elu pole aega , ainus asi kus mu meli 
 särab nii kaua kuni on sees , 
 ei tunne end kui õpetand , kuid tead mida teed hulluks 
 mind ei huvita kummer , näed välja mul ajal , 
 kui palju sõnu . ma tahan rebele 
 sest keelduda raha , miski wppeck on ? 
 su vingu ona taga , lase ma armastan 
 rohenika ja suvepäev saadik saan ma oma maale 
 mul on meil plaan , hull minna ? 
 see venekski sinu kund on niimoodi 
 ei saa jätkata 
 neid asju ette 
 kuigi hellad või hoopis tuleb 
 ja nagu jeeli ta leiab , just saab kuis said 
 mind valusad toimunud , nad kokkuvalged 
 see valksteel liialt minek ka piisavalt 
 siis ma kaddata saan kõik külast , kõik on juba praegu tarkus 
